### Environment Setup

Imports

In [1]:
## This cell will install dask If not installed yet.
# If so please run this notebook again as soon as this cell
# finishes executing. 

try:
  import dask.dataframe as dd   
except ImportError:
  !pip install "dask[complete]" --upgrade
  print("dask installed\n\nPlease RUN this notebook again!")

  # Restarting runtime
  import os
  os.kill(os.getpid(), 9)

#----

#import dask.array as da
from dask.distributed import Client
from zipfile import ZipFile
import os, shutil

Montando o Google Drive

In [2]:
## ATENÇÃO! Aparecerá um link externo seguro provido automaticamente pelo Google Colab para que seja 
## gerado o código de autenticação necessário para o acesso ao Drive.

if not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


Setup dask client

In [3]:
# Dask client setup (Colab RAM)
client = Client()
client

Client Scheduler: tcp://127.0.0.1:45243 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 12.72 GiB


Criando pasta para armazenamento dos dados

In [4]:
if not os.path.exists('/content/DADOS/'):
  !mkdir '/content/DADOS/'

### Data Gathering

Unzipping data folder

In [5]:
def unzip_csv(ano):
  if not isinstance(ano, str):
    ano = str(ano)
  
  zipdata_path = f'/content/drive/MyDrive/Colab Notebooks/IC_Data_Science_ENEM/Dados/microdados_enem_{ano}.zip'
  target_path  = f'/content/DADOS/MICRODADOS_ENEM_{ano}.csv'
  source_path  = f'DADOS/MICRODADOS_ENEM_{ano}.csv'

  if os.path.exists(target_path):
    return True

  # Os dados do ano de 2017 foram compactados de forma diferente:
  if ano == '2017':
    source_path = f'Microdados Enem 2017/{source_path}'

  # Copiando apenas o arquivo .csv para a pasta destino sem todo o caminho
  # herdado do diretório
  with ZipFile(zipdata_path, 'r') as z:
    source = z.open(source_path)
    target = open(target_path, 'wb')

    with source, target:
      shutil.copyfileobj(source, target)

In [6]:
ano = 2017
unzip_csv(ano)

Reading data into dask dataframe

In [7]:
data_path      = f'/content/DADOS/MICRODADOS_ENEM_{str(ano)}.csv'
temp_save_path = f'/content/DADOS/MICRODADOS_ENEM_{str(ano)}.pq'

Informações sobre o dataframe

In [8]:
%%time
if not os.path.exists(temp_save_path):
  # read from csv file
  df = dd.read_csv(data_path, sep=';', encoding='latin-1',
                   dtype={'NU_IDADE'      : 'float64',
                          'TP_PRESENCA_CH': 'float64',
                          'TP_PRESENCA_CN': 'float64',
                          'TP_PRESENCA_LC': 'float64',
                          'TP_PRESENCA_MT': 'float64',
                          'Q005': float})
  # converto to parquet file
  df.to_parquet(temp_save_path)

# read parquet file (more efficient)
df = dd.read_parquet(temp_save_path)
print(df.head())

   NU_INSCRICAO  NU_ANO  CO_MUNICIPIO_RESIDENCIA  ... Q025  Q026 Q027
0  170003336736    2017                  3503208  ...    B     A    A
1  170003333545    2017                  5002902  ...    A     A    A
2  170001663644    2017                  3550308  ...    B     A    A
3  170001663645    2017                  4209300  ...    B     A    A
4  170001663646    2017                  2704302  ...    B     A    A

[5 rows x 137 columns]
CPU times: user 8.76 s, sys: 986 ms, total: 9.75 s
Wall time: 2min 31s


In [9]:
%%time
# Dataframe dimensions
print(f'Rows: {len(df):,}\nColumns: {df.shape[1]}\n')

Rows: 6,731,341
Columns: 137

CPU times: user 377 ms, sys: 36.4 ms, total: 414 ms
Wall time: 1.16 s


In [10]:
%%time
print(f'Número de partições: {df.npartitions}')

Número de partições: 64
CPU times: user 203 µs, sys: 0 ns, total: 203 µs
Wall time: 553 µs


In [11]:
%%time
# Partition Average Sizes 
df.map_partitions(len).compute().head()   # Takes longer than other cells due to compute() method

CPU times: user 2.3 s, sys: 263 ms, total: 2.56 s
Wall time: 35.2 s


0    102146
1    101609
2    100061
3    100296
4     97707
dtype: int64

In [12]:
%%time
# Accessing partition 1 (second one)
df.get_partition(1).head()

CPU times: user 65.8 ms, sys: 8.4 ms, total: 74.2 ms
Wall time: 789 ms


,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,IN_VISAO_MONOCULAR,...,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027
0,170001692613,2017,5201405,Aparecida de Goiânia,52,GO,20.0,F,1.0,2,1,5201405.0,Aparecida de Goiânia,52.0,GO,1,1,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,CEBCADECEBADEBDBDEABDBCDAABECD.EACBECADBACEBD,0,DEEBDABCBBEDDCBABCADECEBAADAAECDCBCCDACDEEAAE,CDDECADBEABDBEDAECAEBDAEBAEDBDBBAECDAEBCCCCDE,DDCDEEDBEEBDAEDAABCECDAEBADEDEDBBBDEABBCCABAAE...,ADBCCECBBDBAEBBDDDABDCCDEDECBEACDAEAABBACEECD,1.0,120.0,120.0,140.0,140.0,100.0,620.0,C,D,D,B,3.0,B,A,B,B,A,B,B,B,B,A,B,A,A,B,A,A,C,A,A,A,A,A
1,170001692608,2017,2507507,João Pessoa,25,PB,40.0,M,NaN,2,1,1303809.0,São Gabriel da Cachoeira,13.0,AM,1,11,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,DAECDDCBBDBBACDADCBEEBCBCCBEABDCEACDBAEABDDDC,1,EEAAECDCCDADAADECDCBCDAAEDEEBABCEDDCBEBAABCBB,CDDECADBEABDBEDAECAEBDAEBAEDBDBBAECDAEBCCCCDE,DDCDEEDBEEBDAEDAABCECDAEBADEDEDBBBDEABBCCABAAE...,BBDBADBEECDCCECAEBBDDEDDCCBBACADAEACEADABDECB,1.0,100.0,100.0,100.0,80.0,20.0,400.0,E,E,D,F,2.0,B,A,C,C,A,B,B,A,B,A,B,A,A,B,A,A,C,A,B,B,A,A
2,170001692683,2017,3304557,Rio de Janeiro,33,RJ,18.0,F,0.0,2,1,3304557.0,Rio de Janeiro,33.0,RJ,2,0,2,1.0,0,33075166.0,3304557.0,Rio de Janeiro,33.0,RJ,2.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,...,EDBCDCADCEDAEABBDCCDCADBCBAACBCAECEADCCAADBDB,0,EEAAECDCCDADAADECDCBCDAAEDEEBABCEDDCBEBAABCBB,CDEAEECAEBDBDBBAECDAEBCCCDAEBEABDBEDAADBCDDEC,EDDCDBEEDECCEBDAEDAEDAABEDBBADEDEDDABAABBDCBEA...,BBDBADBEECDCCECAEBBDDEDDCCBBACADAEACEADABDECB,1.0,160.0,120.0,120.0,100.0,120.0,620.0,E,E,B,B,5.0,C,A,B,B,A,B,B,A,B,A,B,A,A,B,A,A,E,B,B,B,B,A
3,170001686658,2017,2202083,Cajueiro da Praia,22,PI,18.0,F,0.0,1,1,2202083.0,Cajueiro da Praia,22.0,PI,1,2,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,None,0,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E,B,F,F,4.0,C,A,B,C,A,A,B,A,A,A,A,A,A,B,B,A,D,A,B,B,A,A
4,170001692605,2017,2412005,São Gonçalo do Amarante,24,RN,24.0,M,NaN,3,1,2412005.0,São Gonçalo do Amarante,24.0,RN,1,4,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,None,1,None,CDDECADBEABDBEDAECAEBDAEBAEDBDBBAECDAEBCCCCDE,DDCDEEDBEEBDAEDAABCECDAEBADEDEDBBBDEABBCCABAAE...,None,1.0,100.0,120.0,80.0,80.0,80.0,460.0,B,D,B,B,5.0,C,A,B,D,A,A,B,A,A,A,B,A,A,B,A,A,D,A,B,B,A,A


Com *dask*, sempre que precisar utilizar alguma funcionalidade suportada apenas por *Pandas*, usa-se *compute()* - ou *len(dask_df)*, como acima - que "varre" todas as partições em que os dados foram armazenados. O tempo decorrido é inversamente proporcional ao espaço em memória RAM e ao número de "trabalhadores" utilizados (por padrão, igual ao número de núcleos presente no processador).

No Google Colab:

1.   Memória RAM alocável: **12,72 GB**

2.   Número de núcleos (*workers*) disponível: **2 núcleos**

Algumas operações podem tomar um certo tempo devido ao número relativamente baixo de núcleos no amiente virtual. Entretanto, é apreciado que seja possível ter todos os dados sem que a memória RAM seja sobrecarregada ou que sejam excluídas colunas antes mesmo de serem analisadas.

Para isso, tem-se 50 partições com aproximadamente 100 mil linhas de dados.



### Pré-processamento de Dados

*Roadmap*:

Manipulando colunas

*   TP_PRESENCA: 0 if any 0 in TP_PRESENCAs else 1  #ok
*   NU_NOTA_MEDIA: média das NU_NOTAs               #ok
*   Atendimento Especializado/Específico            #ok

Excluindo colunas

*   Colunas sobre escola com proporção de valores nulos muito alta (>50%?) #ok
*   Respostas, gabaritos, notas de competências e códigos de provas       #ok
*   Pertencentes a IN_SEM_RECURSO     #ok



#### Manipulando colunas

Agrupando dados espalhados em múltiplas colunas no menor número possível de counas a fim de reduzir as dimensões da base de dados.

**Coluna de Média de Notas**

Para dar simplicidade e objetividade ao algoritmo posteriormente, é interessante utilizar apenas uma colunas como alvo com relação às notas. Portanto, as notas serão representadas pela média de cada uma das 5 (cinco) áeras do conhecimento.

Além disso, todas as demais colunas relacionadas às notas (notas de competência na Redação, nota em cada cadernos de prova, folhas-resposta e gabaritos)

In [13]:
cols_notas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']
df = df.assign(NU_NOTA_MEDIA = df[cols_notas].mean(axis=1, skipna=False))   
# Se quiser saber somente quantos não foram em nenhum dia, basta tirar o parâmetro skipna

**Coluna de Presença**

Se o inscrito constar ausência em algum ou em todos os dias será 
registrado como *TP_PRESENCA: False*. Dessa forma, a definição de ausente/desistente é obtida com maior clareza a partir de uma única coluna. Aquelas que serviram de base para se obter TP_PRESENCA serão excluídas.

In [14]:
# Forma vetorizada - desempenho maior
#df['TP_PRESENCA'] = (
#    (df.TP_PRESENCA_CH==1) & (df.TP_PRESENCA_CN==1) & (df.TP_PRESENCA_LC==1) & (df.TP_PRESENCA_MT==1) & (df.TP_STATUS_REDACAO==1)
#)

df['TP_PRESENCA'] = ~df['NU_NOTA_MEDIA'].isna()
df['TP_PRESENCA'] = df['TP_PRESENCA'].astype(int)

In [15]:
%%time
df['TP_PRESENCA'].value_counts().compute()  # o parâmetro normalize=True não tem suporte para Dask

CPU times: user 2.45 s, sys: 282 ms, total: 2.73 s
Wall time: 38.7 s


1    4426755
0    2304586
Name: TP_PRESENCA, dtype: int64

Em *pandas*, o método *value_counts* possui um parâmetro chamado *normalize* que retorna a frequência de cada valor. Em *Dask*, entretanto, não é suportado.

Sendo assim, realizando um cálculo manual. tem-se que a **razão de desistência ou ausência, em qualquer das cinco provas, é de 27,34%** 

**Coluna de Atendimento Especializado**

O INEP oferece opções de atendimento especializado que são dispostas em 13 colunas. Como compreendem uma pequena parte da composição, serão agrupadas em uma só coluna que indicará se o inscrito requisitou ou não algum atendimento especializado para o Exame.

In [16]:
cols_especial = ['IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ', 'IN_DEFICIENCIA_AUDITIVA',
                 'IN_SURDO_CEGUEIRA', 'IN_DEFICIENCIA_FISICA', 'IN_DEFICIENCIA_MENTAL',
                 'IN_DEFICIT_ATENCAO', 'IN_DISLEXIA', 'IN_DISCALCULIA', 'IN_AUTISMO',
                 'IN_VISAO_MONOCULAR', 'IN_OUTRA_DEF']

df = df.assign(IN_ATEND_ESPECIALIZADO = df.loc[:, cols_especial].max(1))

**Coluna de Atendimento Específico**

O Inep oferece opções de atendimento a condições específicas que são dispostas em 4 colunas. Como compreendem uma pequena parte da composição, serão agrupadas em uma só coluna que indicará se o inscrito requisitou ou não algum atendimento específico para o Exame.

In [17]:
cols_especifico = ['IN_GESTANTE', 'IN_LACTANTE', 'IN_IDOSO', 'IN_ESTUDA_CLASSE_HOSPITALAR']
df = df.assign(IN_ATEND_ESPECIFICO = df.loc[:, cols_especifico].max(1))

In [18]:
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,IN_VISAO_MONOCULAR,...,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,NU_NOTA_MEDIA,TP_PRESENCA,IN_ATEND_ESPECIALIZADO,IN_ATEND_ESPECIFICO
0,170003336736,2017,3503208,Araraquara,35,SP,29.0,F,0.0,1,1,3503208.0,Araraquara,35.0,SP,1,11,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,DEDCDBEEDEECABAAEDDCBDEDBAEABBBDEDBBCCDAEBCAAB...,EECDCCECBDBBDBAEBADBECBDDABDEDEAADCCDAEACBBAC,1.0,140.0,120.0,120.0,100.0,80.0,560.0,E,D,C,B,3.0,D,A,B,C,A,B,B,A,C,A,B,A,A,C,B,B,C,B,B,B,A,A,536.28,1,0,0
1,170003333545,2017,5002902,Cassilândia,50,MS,22.0,F,0.0,1,1,5002902.0,Cassilândia,50.0,MS,1,5,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,E,B,B,4.0,C,A,B,D,B,B,B,A,A,A,B,A,A,B,A,A,C,B,A,A,A,A,NaN,0,0,0
2,170001663644,2017,3550308,São Paulo,35,SP,38.0,F,0.0,1,1,3550308.0,São Paulo,35.0,SP,1,11,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,A,F,B,4.0,G,A,B,C,A,A,B,A,B,A,B,A,A,B,A,A,C,A,B,B,A,A,NaN,0,0,0
3,170001663645,2017,4209300,Lages,42,SC,35.0,F,0.0,1,1,4209300.0,Lages,42.0,SC,1,6,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,DDCDEEDBEEBDAEDAABCECDAEBADEDEDBBBDEABBCCABAAE...,EECDCCECBDBBDBAEBADBECBDDABDEDEAADCCDAEACBBAC,1.0,140.0,120.0,140.0,100.0,80.0,580.0,E,D,B,B,4.0,D,A,B,D,B,A,B,A,B,A,B,A,B,C,A,B,D,A,B,B,A,A,589.90,1,0,0
4,170001663646,2017,2704302,Maceió,27,AL,40.0,M,0.0,3,1,2704302.0,Maceió,27.0,AL,1,11,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,DDCDEEDBEEBDAEDAABCECDAEBADEDEDBBBDEABBCCABAAE...,CCECEECDADBBDBBDBAEBDDABECBDCCDEDBBACAEADAEAC,1.0,140.0,120.0,120.0,120.0,80.0,580.0,A,B,B,F,3.0,C,A,B,B,A,B,B,A,A,A,A,A,A,B,B,A,C,A,C,B,A,A,557.34,1,0,0


#### Excluindo colunas

**Colunas de Presenças, Notas, Atendimento Especializado e Antendimento Específico**

Após ter reunidas as informações de tantas colunas em um número bem menor, pode-se, agora, excluír essas colunas sem que se perca informação relevante e, assim, reduzir as dimensões da base de dados.

In [19]:
cols_presenca = ['TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'TP_STATUS_REDACAO']
cols_notas    = ['CO_PROVA_CN', 'TP_LINGUA', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2',
                 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC',
                 'TX_RESPOSTAS_MT', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT']
cols_to_delete = cols_presenca + cols_notas + cols_especial + cols_especifico
#print(cols_to_delete)

df = df.drop(cols_to_delete, axis=1)

In [20]:
# Por alguma razão, é necessário excluir estas colunas separadamente
# Se excluir junto com as demais, os valores da média de notas se alteram       ## Checar motivo
df = df.drop(['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT'], axis=1)

In [21]:
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_SEM_RECURSO,IN_BRAILLE,IN_AMPLIADA_24,IN_AMPLIADA_18,IN_LEDOR,IN_ACESSO,IN_TRANSCRICAO,IN_LIBRAS,IN_LEITURA_LABIAL,IN_MESA_CADEIRA_RODAS,IN_MESA_CADEIRA_SEPARADA,IN_APOIO_PERNA,...,IN_SALA_ACOMPANHANTE,IN_MOBILIARIO_ESPECIFICO,IN_MATERIAL_ESPECIFICO,IN_NOME_SOCIAL,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,NU_NOTA_MEDIA,TP_PRESENCA,IN_ATEND_ESPECIALIZADO,IN_ATEND_ESPECIFICO
0,170003336736,2017,3503208,Araraquara,35,SP,29.0,F,0.0,1,1,3503208.0,Araraquara,35.0,SP,1,11,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3503208,Araraquara,35,SP,560.0,E,D,C,B,3.0,D,A,B,C,A,B,B,A,C,A,B,A,A,C,B,B,C,B,B,B,A,A,536.28,1,0,0
1,170003333545,2017,5002902,Cassilândia,50,MS,22.0,F,0.0,1,1,5002902.0,Cassilândia,50.0,MS,1,5,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,5002902,Cassilândia,50,MS,NaN,C,E,B,B,4.0,C,A,B,D,B,B,B,A,A,A,B,A,A,B,A,A,C,B,A,A,A,A,NaN,0,0,0
2,170001663644,2017,3550308,São Paulo,35,SP,38.0,F,0.0,1,1,3550308.0,São Paulo,35.0,SP,1,11,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3550308,São Paulo,35,SP,NaN,A,A,F,B,4.0,G,A,B,C,A,A,B,A,B,A,B,A,A,B,A,A,C,A,B,B,A,A,NaN,0,0,0
3,170001663645,2017,4209300,Lages,42,SC,35.0,F,0.0,1,1,4209300.0,Lages,42.0,SC,1,6,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,4209300,Lages,42,SC,580.0,E,D,B,B,4.0,D,A,B,D,B,A,B,A,B,A,B,A,B,C,A,B,D,A,B,B,A,A,589.90,1,0,0
4,170001663646,2017,2704302,Maceió,27,AL,40.0,M,0.0,3,1,2704302.0,Maceió,27.0,AL,1,11,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2704302,Maceió,27,AL,580.0,A,B,B,F,3.0,C,A,B,B,A,B,B,A,A,A,A,A,A,B,B,A,C,A,C,B,A,A,557.34,1,0,0


**Colunas de Recursos Especiais e Específicos**

O Inep oferece, ainda, a opção do inscrito solicitar algum recurso especializado ou específico, como prova em Braille, protetor auricular, intérpretes, leito para realizar prova deitado, entre outros.

Existe, porém, na base de dados fornecida pelo MEC/Inep, a coluna *IN_SEM_RECURSO* que já informa se o inscrito solicitou ou não algum desses recursos e, portanto, não foi necessário reunir os dados de todas as colunas relacionadas a esse indicador para uma única coluna como foi feito acima com os outros indicadores.

Desta forma, entende-se por necessário manter apensas a coluna *IN_SEM_RECURSO* e excluir as outras, visto que nela já estão contidas todas as informações relevantes para o estudo.

In [22]:
cols_recurso = ['IN_BRAILLE', 'IN_AMPLIADA_24', 'IN_AMPLIADA_18', 'IN_LEDOR', 'IN_ACESSO',
                'IN_TRANSCRICAO', 'IN_LIBRAS', 'IN_TEMPO_ADICIONAL', 'IN_LEITURA_LABIAL',
                'IN_MESA_CADEIRA_RODAS', 'IN_MESA_CADEIRA_SEPARADA', 'IN_APOIO_PERNA',
                'IN_GUIA_INTERPRETE', 'IN_COMPUTADOR', 'IN_CADEIRA_ESPECIAL',
                'IN_CADEIRA_CANHOTO', 'IN_CADEIRA_ACOLCHOADA', 'IN_PROVA_DEITADO',
                'IN_MOBILIARIO_OBESO', 'IN_LAMINA_OVERLAY', 'IN_PROTETOR_AURICULAR',
                'IN_MEDIDOR_GLICOSE', 'IN_MAQUINA_BRAILE', 'IN_SOROBAN', 'IN_MARCA_PASSO',
                'IN_SONDA', 'IN_MEDICAMENTOS', 'IN_SALA_INDIVIDUAL', 'IN_SALA_ESPECIAL',
                'IN_SALA_ACOMPANHANTE', 'IN_MOBILIARIO_ESPECIFICO', 'IN_MATERIAL_ESPECIFICO',
                'IN_NOME_SOCIAL']

df = df.drop(cols_recurso, axis=1, errors='ignore')

In [23]:
df.head()   # Já é possível ver todas as colunas do dataframe sem cortes/pulos

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_SEM_RECURSO,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,NU_NOTA_MEDIA,TP_PRESENCA,IN_ATEND_ESPECIALIZADO,IN_ATEND_ESPECIFICO
0,170003336736,2017,3503208,Araraquara,35,SP,29.0,F,0.0,1,1,3503208.0,Araraquara,35.0,SP,1,11,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,3503208,Araraquara,35,SP,560.0,E,D,C,B,3.0,D,A,B,C,A,B,B,A,C,A,B,A,A,C,B,B,C,B,B,B,A,A,536.28,1,0,0
1,170003333545,2017,5002902,Cassilândia,50,MS,22.0,F,0.0,1,1,5002902.0,Cassilândia,50.0,MS,1,5,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,5002902,Cassilândia,50,MS,NaN,C,E,B,B,4.0,C,A,B,D,B,B,B,A,A,A,B,A,A,B,A,A,C,B,A,A,A,A,NaN,0,0,0
2,170001663644,2017,3550308,São Paulo,35,SP,38.0,F,0.0,1,1,3550308.0,São Paulo,35.0,SP,1,11,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,3550308,São Paulo,35,SP,NaN,A,A,F,B,4.0,G,A,B,C,A,A,B,A,B,A,B,A,A,B,A,A,C,A,B,B,A,A,NaN,0,0,0
3,170001663645,2017,4209300,Lages,42,SC,35.0,F,0.0,1,1,4209300.0,Lages,42.0,SC,1,6,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,4209300,Lages,42,SC,580.0,E,D,B,B,4.0,D,A,B,D,B,A,B,A,B,A,B,A,B,C,A,B,D,A,B,B,A,A,589.90,1,0,0
4,170001663646,2017,2704302,Maceió,27,AL,40.0,M,0.0,3,1,2704302.0,Maceió,27.0,AL,1,11,1,NaN,0,NaN,NaN,None,NaN,None,NaN,NaN,NaN,0,2704302,Maceió,27,AL,580.0,A,B,B,F,3.0,C,A,B,B,A,B,B,A,A,A,A,A,A,B,B,A,C,A,C,B,A,A,557.34,1,0,0


**Verificando a frequência de valores nulos**

In [24]:
%%time
# Might take a little long - 2x compute() and 1x len(dask_df)
df_columns_with_nan = df[df.columns[df.isna().any().compute()]]  # Parte do dataframe original apenas com as colunas que contem valores nulos
df_nan = df_columns_with_nan.isna().sum().compute().to_frame()   # Dataframe com a soma de valores nulos de cada coluna de df_withnan

df_nan.rename(columns={0: 'Quantidade'}, inplace=True)
df_nan['Porcentagem'] = 100*df_nan['Quantidade']/len(df_columns_with_nan)

CPU times: user 11.6 s, sys: 1.06 s, total: 12.7 s
Wall time: 2min 58s


In [25]:
df_nan

,Quantidade,Porcentagem
NU_IDADE,101,0.001500
TP_ESTADO_CIVIL,271641,4.035466
CO_MUNICIPIO_NASCIMENTO,229773,3.413480
NO_MUNICIPIO_NASCIMENTO,229773,3.413480
CO_UF_NASCIMENTO,229773,3.413480
SG_UF_NASCIMENTO,229773,3.413480
TP_ENSINO,4945436,73.468808
CO_ESCOLA,4945420,73.468570
CO_MUNICIPIO_ESC,4945427,73.468674
NO_MUNICIPIO_ESC,4945427,73.468674


Decidiu-se por excluir aqueles que possuírem valores nulos em NU_IDADE, TP_ESTADO_CIVIL e às questões do formulário sócioeconômico.

In [26]:
subset = ['NU_IDADE', 'TP_ESTADO_CIVIL']
subset += [f'Q0{i:02}' for i in range(1,28)]
#print(subset)

df = df.dropna(how='any', subset=subset)

Porcentagem de dados faltantes a respeito das escolas em cada estado brasileiro

In [27]:
import pandas as pd
import plotly.express as px

In [28]:
%%time
esc_uf = df.groupby('SG_UF_PROVA')['SG_UF_ESC']
esc_uf.count().head(27)

CPU times: user 4.09 s, sys: 431 ms, total: 4.52 s
Wall time: 1min 3s


In [29]:
%%time
ausencia_esc_uf_pct = pd.crosstab(
    df['SG_UF_PROVA'],df['SG_UF_ESC'].fillna('NaN'), colnames=['UF_ESC'], rownames=['UF_PROVA']
)

CPU times: user 27.6 s, sys: 3.65 s, total: 31.3 s
Wall time: 6min 21s


In [30]:
ausencia_esc_uf_pct  = ausencia_esc_uf_pct.apply(lambda x: x/x.sum()*100, axis=1)

stacked_esc = ausencia_esc_uf_pct.reset_index().rename(columns={'NaN':'value'})
stacked_esc = stacked_esc.sort_values(by='value', ascending=False)

In [31]:
fig = px.bar(
    stacked_esc, x='UF_PROVA', y='value',
    text = 'value',
    labels={
        'value': 'Dados Ausentes [%]',
        'UF_PROVA': 'Estado'},
    height=600, width=1200,
    title=f'Percentual de Dados Ausentes sobre as Escolas por Estado do Brasil no ENEM {str(ano)}'
    )
#fig.update_layout(
#    font_family="Courier New",
#    font_color="blue",
#    title_font_family="Times New Roman",
#    title_font_color="red"
#)
fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}    
)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.show()

É possível observar que diversas informações sobre as escolas estão com uma taxa de omissão bem alta (aproximadamente 73% dos dados totais estão faltantes).

Como não há uma notável diferença de proporção entre os estados, não é possível inferir nenhuma relação tampouco tomar qualquer decisão baseada nesses dados, já que fica muito difícil de substituí-los ou deletar os inscritos sem esses dados.

In [32]:
columns_to_drop = ['CO_ESCOLA', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC',
                   'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'TP_ENSINO']
df = df.drop(labels=columns_to_drop, axis=1, errors='ignore')

In [33]:
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,IN_SEM_RECURSO,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,NU_NOTA_MEDIA,TP_PRESENCA,IN_ATEND_ESPECIALIZADO,IN_ATEND_ESPECIFICO
0,170003336736,2017,3503208,Araraquara,35,SP,29.0,F,0.0,1,1,3503208.0,Araraquara,35.0,SP,1,11,1,0,0,3503208,Araraquara,35,SP,560.0,E,D,C,B,3.0,D,A,B,C,A,B,B,A,C,A,B,A,A,C,B,B,C,B,B,B,A,A,536.28,1,0,0
1,170003333545,2017,5002902,Cassilândia,50,MS,22.0,F,0.0,1,1,5002902.0,Cassilândia,50.0,MS,1,5,1,0,0,5002902,Cassilândia,50,MS,NaN,C,E,B,B,4.0,C,A,B,D,B,B,B,A,A,A,B,A,A,B,A,A,C,B,A,A,A,A,NaN,0,0,0
2,170001663644,2017,3550308,São Paulo,35,SP,38.0,F,0.0,1,1,3550308.0,São Paulo,35.0,SP,1,11,1,0,0,3550308,São Paulo,35,SP,NaN,A,A,F,B,4.0,G,A,B,C,A,A,B,A,B,A,B,A,A,B,A,A,C,A,B,B,A,A,NaN,0,0,0
3,170001663645,2017,4209300,Lages,42,SC,35.0,F,0.0,1,1,4209300.0,Lages,42.0,SC,1,6,1,0,0,4209300,Lages,42,SC,580.0,E,D,B,B,4.0,D,A,B,D,B,A,B,A,B,A,B,A,B,C,A,B,D,A,B,B,A,A,589.90,1,0,0
4,170001663646,2017,2704302,Maceió,27,AL,40.0,M,0.0,3,1,2704302.0,Maceió,27.0,AL,1,11,1,0,0,2704302,Maceió,27,AL,580.0,A,B,B,F,3.0,C,A,B,B,A,B,B,A,A,A,A,A,A,B,B,A,C,A,C,B,A,A,557.34,1,0,0


Com relação à **localidade de nascimento**, há duas opções viáveis:

1.   Caso uma parte considerável dos inscritos possuam UF e/ou código de município de nascimento diferente do de residência, talvez possa ser interessante guardar essa informação em uma coluna estilo "TP_MOVIMENTO" e cabe alguma análise de alguma diferença de desempenho ou correlação com algum outro indicador, como renda familiar ou UF de destino, por exemplo.

2.   Caso contrário, considera-se que a localidade permanece a mesma e pode ser desconsiderada qualquer coluna relacionada às localidades de nascimento e residência, tomando como base de geolocalização apenas a UF onde foi realizada a prova.

In [34]:
%%time
# Proporção de pessoas que mudaram de estado
n = len(df[df['SG_UF_NASCIMENTO']!=df['SG_UF_RESIDENCIA']])
print('Pessoas que se mudaram de estado desde que nasceram:\nQuantidade: {}\t{:.2f}%'.format(n, 100*n/len(df)))

Pessoas que se mudaram de estado desde que nasceram:
Quantidade: 994234	15.39%
CPU times: user 8.82 s, sys: 2.91 s, total: 11.7 s
Wall time: 2min 13s


In [35]:
%%time
# Proporção de pessoas que mudaram de município
n = len(df[df['CO_MUNICIPIO_NASCIMENTO']!=df['CO_MUNICIPIO_RESIDENCIA']])
print('Pessoas que se mudaram de cidade desde que nasceram:\nQuantidade: {}\t{:.2f}%'.format(n, 100*n/len(df)))

Pessoas que se mudaram de cidade desde que nasceram:
Quantidade: 2775841	42.97%
CPU times: user 8.28 s, sys: 823 ms, total: 9.1 s
Wall time: 2min 12s


Os valores nulos serão substituídos pela maior frequência da maior, ou seja, manter-se-á a mesma localidade, mas observando que há uma proporção significativa de inscritos que se mudaram de cidade e Unidade da Federação.

Por conta disso, talvez seja interessante guardar essa informação para eventuais análises.

In [36]:
df['CO_MUNICIPIO_NASCIMENTO'] = df['CO_MUNICIPIO_NASCIMENTO'].fillna(value=df['CO_MUNICIPIO_PROVA'], axis=0)
df['NO_MUNICIPIO_NASCIMENTO'] = df['NO_MUNICIPIO_NASCIMENTO'].fillna(value=df['NO_MUNICIPIO_PROVA'], axis=0)
df['CO_UF_NASCIMENTO'] = df['CO_UF_NASCIMENTO'].fillna(value=df['CO_UF_PROVA'], axis=0)
df['SG_UF_NASCIMENTO'] = df['SG_UF_NASCIMENTO'].fillna(value=df['SG_UF_PROVA'], axis=0)

In [37]:
df['TP_UF_MUDANCA'] = (df['CO_UF_NASCIMENTO']!=df['CO_UF_PROVA']) | (df['CO_UF_RESIDENCIA']!=df['CO_UF_NASCIMENTO'])
df['TP_UF_MUDANCA'] = df['TP_UF_MUDANCA'].astype(int)
df['TP_MUNICIPIO_MUDANCA'] = (df['CO_MUNICIPIO_NASCIMENTO']!=df['CO_MUNICIPIO_PROVA']) | (df['CO_MUNICIPIO_RESIDENCIA']!=df['CO_MUNICIPIO_NASCIMENTO'])
df['TP_MUNICIPIO_MUDANCA'] = df['TP_MUNICIPIO_MUDANCA'].astype(int)

In [38]:
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,IN_SEM_RECURSO,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,NU_NOTA_MEDIA,TP_PRESENCA,IN_ATEND_ESPECIALIZADO,IN_ATEND_ESPECIFICO,TP_UF_MUDANCA,TP_MUNICIPIO_MUDANCA
0,170003336736,2017,3503208,Araraquara,35,SP,29.0,F,0.0,1,1,3503208.0,Araraquara,35.0,SP,1,11,1,0,0,3503208,Araraquara,35,SP,560.0,E,D,C,B,3.0,D,A,B,C,A,B,B,A,C,A,B,A,A,C,B,B,C,B,B,B,A,A,536.28,1,0,0,0,0
1,170003333545,2017,5002902,Cassilândia,50,MS,22.0,F,0.0,1,1,5002902.0,Cassilândia,50.0,MS,1,5,1,0,0,5002902,Cassilândia,50,MS,NaN,C,E,B,B,4.0,C,A,B,D,B,B,B,A,A,A,B,A,A,B,A,A,C,B,A,A,A,A,NaN,0,0,0,0,0
2,170001663644,2017,3550308,São Paulo,35,SP,38.0,F,0.0,1,1,3550308.0,São Paulo,35.0,SP,1,11,1,0,0,3550308,São Paulo,35,SP,NaN,A,A,F,B,4.0,G,A,B,C,A,A,B,A,B,A,B,A,A,B,A,A,C,A,B,B,A,A,NaN,0,0,0,0,0
3,170001663645,2017,4209300,Lages,42,SC,35.0,F,0.0,1,1,4209300.0,Lages,42.0,SC,1,6,1,0,0,4209300,Lages,42,SC,580.0,E,D,B,B,4.0,D,A,B,D,B,A,B,A,B,A,B,A,B,C,A,B,D,A,B,B,A,A,589.90,1,0,0,0,0
4,170001663646,2017,2704302,Maceió,27,AL,40.0,M,0.0,3,1,2704302.0,Maceió,27.0,AL,1,11,1,0,0,2704302,Maceió,27,AL,580.0,A,B,B,F,3.0,C,A,B,B,A,B,B,A,A,A,A,A,A,B,B,A,C,A,C,B,A,A,557.34,1,0,0,0,0


#### Colunas "Extras"

Algumas colunas foram formatadas pelo Inep de forma a representar dados não informados com "1". Por isso, muitas não são detectadas com *isna()* e é preciso abordá-las individualmente de acordo com o Dicionário dos Microdados fornecido também pelo Inep.

Entre as que ainda estão presentes no Dataframe, constituem esse grupo:

*   TP_COR_RACA
*   TP_NACIONALIDADE
*   TP_ANO_CONCLUIU
*   TP_ESCOLA

Sendo assim, a começar por **TP_COR_RACA**, realiza-se uma análise de cada uma dessas colunas para ser decidido como serão tratados os dados que os inscritos escolheram não responder. Para isso, tenha-se que "0 = Não declarado"

In [39]:
df['TP_COR_RACA'].value_counts().compute()

3    3028930
1    2269816
2     858155
4     148674
0     112473
5      41552
Name: TP_COR_RACA, dtype: int64

É possível observar que aqueles que escolheram não declarar ou não se sentiram representados pelas outras alternativas não representam, necessariamente, um grupo de dados faltantes. É possível tomar como um grupo como os outros.

A coluna deve, então, permanecer da mesma forma.

Já com  **TP_NACIONALIDADE**, dado que "0 = Não informado", tem-se:


In [40]:
df['TP_NACIONALIDADE'].value_counts().compute()

1    6239420
2     194614
4      15685
3       7851
0       2030
Name: TP_NACIONALIDADE, dtype: int64

A proporção daqueles que não informaram não representam relevância perante os outros dados. Como a grande maioria é "1 = Brasileiro(a)", substituir-se-á todos zeros por um.

In [41]:
df['TP_NACIONALIDADE'] = df['TP_NACIONALIDADE'].replace(0,1)

Para a **coluna TP_ANO_CONCLUIU**, com "0 = Não informado", tem-se:

In [42]:
df['TP_ANO_CONCLUIU'].value_counts().compute()

0     2394065
1      839200
11     741924
2      590003
3      441989
4      339474
5      278777
6      213557
7      198094
8      165545
9      133932
10     123040
Name: TP_ANO_CONCLUIU, dtype: int64

Aqueles que não concluíram o Ensino Médio ainda também marcaram "0" nesse item do formulário. Apesar de haver alguns inscritos que marcaram desta forma mesmo assinalando no item anterior (TP_ST_CONCLUSAO) que já haviam concluído o Ensino Médio, viu desnecessário distinguí-los, visto que representam uma quantidade pequena com relação ao todo.

Além disso, aproveita-se para fazer a seguinte alocação:

*   Serão agrupados todos aqueles que concluíram o Ensino Médio a 10 (dez) anos ou mais antes de realizarem o respectivo Exame, ou seja, os valores da coluna TP_ANO_CONCLUIU maiores que 10 serão substituídos por 10.

Desta forma, obtem-se uma clusterização mais "arredondada" mas que mantém a informação relevante.

In [43]:
condition = df['TP_ANO_CONCLUIU'] > 10
df['TP_ANO_CONCLUIU'] = df['TP_ANO_CONCLUIU'].mask(condition, 10)

Com relação à **coluna TP_ESCOLA**, sendo "1 = Não Respondeu": 



In [44]:
df['TP_ESCOLA'].value_counts().compute()

1    4717961
2    1451035
3     286103
4       4501
Name: TP_ESCOLA, dtype: int64

Devido ao número muito grande de inscritos que não responderam, será mantida esta coluna de forma que o grupo que respondeu possa ser agrupado na etapa de Análise Exploratória.

In [45]:
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_TREINEIRO,IN_SEM_RECURSO,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,NU_NOTA_MEDIA,TP_PRESENCA,IN_ATEND_ESPECIALIZADO,IN_ATEND_ESPECIFICO,TP_UF_MUDANCA,TP_MUNICIPIO_MUDANCA
0,170003336736,2017,3503208,Araraquara,35,SP,29.0,F,0.0,1,1,3503208.0,Araraquara,35.0,SP,1,10,1,0,0,3503208,Araraquara,35,SP,560.0,E,D,C,B,3.0,D,A,B,C,A,B,B,A,C,A,B,A,A,C,B,B,C,B,B,B,A,A,536.28,1,0,0,0,0
1,170003333545,2017,5002902,Cassilândia,50,MS,22.0,F,0.0,1,1,5002902.0,Cassilândia,50.0,MS,1,5,1,0,0,5002902,Cassilândia,50,MS,NaN,C,E,B,B,4.0,C,A,B,D,B,B,B,A,A,A,B,A,A,B,A,A,C,B,A,A,A,A,NaN,0,0,0,0,0
2,170001663644,2017,3550308,São Paulo,35,SP,38.0,F,0.0,1,1,3550308.0,São Paulo,35.0,SP,1,10,1,0,0,3550308,São Paulo,35,SP,NaN,A,A,F,B,4.0,G,A,B,C,A,A,B,A,B,A,B,A,A,B,A,A,C,A,B,B,A,A,NaN,0,0,0,0,0
3,170001663645,2017,4209300,Lages,42,SC,35.0,F,0.0,1,1,4209300.0,Lages,42.0,SC,1,6,1,0,0,4209300,Lages,42,SC,580.0,E,D,B,B,4.0,D,A,B,D,B,A,B,A,B,A,B,A,B,C,A,B,D,A,B,B,A,A,589.90,1,0,0,0,0
4,170001663646,2017,2704302,Maceió,27,AL,40.0,M,0.0,3,1,2704302.0,Maceió,27.0,AL,1,10,1,0,0,2704302,Maceió,27,AL,580.0,A,B,B,F,3.0,C,A,B,B,A,B,B,A,A,A,A,A,A,B,B,A,C,A,C,B,A,A,557.34,1,0,0,0,0


Por fim, redefinindo os tipos das colunas 

In [46]:
dtype = {'NU_IDADE': int,
         'TP_ESTADO_CIVIL': int,
         'CO_MUNICIPIO_NASCIMENTO': int,
         'CO_UF_NASCIMENTO': int}
df = df.astype(dtype)

### Salvando Arquivo

Salvando arquivo no formato .parquet para a fase de Análise Exploratória dos Dados e para a preparação dos dados a fim de servirem de entrada aos algoritmos de Machine Learning.

In [47]:
save_path = f'/content/drive/MyDrive/Colab Notebooks/IC_Data_Science_ENEM/Dados/PRE_PROCESSED_ENEM_{str(ano)}.pq'
df.to_parquet(save_path, write_index=None)